In [3]:
import pandas as pd
from src.eventlog_utils import convert_log
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from log_distance_measures.config import EventLogIDs, AbsoluteTimestampType, discretize_to_hour
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.work_in_progress import work_in_progress_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance
from log_distance_measures import earth_movers_distance
from src.res_based_ced import resource_based_circadian_event_distribution_distance
from distance_utils import emd_attributes

In [2]:
case_studies = {
    1: 'Purchasing',
    2: 'Production',
    3: 'Consulta',
    4: 'bpi12',
    5: 'bpi17',
    6: 'sepsis',
    7: 'rtf',
    8: 'bpi19'
}

In [3]:
# chose case study
case_study = case_studies[5]

# Choose if our approach or Sota
our_approach = False
Sota = not(our_approach)
approach = 'SIMOD' if Sota else None
print(f'Case study: {case_study}, our approach = {our_approach}')

Case study: bpi17, our approach = False


In [4]:
log_real = xes_importer.apply(f'data/{case_study}/logTest.xes')
log_real = pm4py.convert_to_dataframe(log_real)

if our_approach:
    log_sim = pd.read_csv(f'simulations/{case_study}/sim_0.csv')
    print('imported log from our approach')

elif Sota:
    
    if approach == 'RIMS':
        log_sim = pd.read_csv(f'RIMS/{case_study}/results/rims/sim.csv')
        del log_sim['st_wip']
        del log_sim['queue']
        del log_sim['st_tsk_wip']
        print('imported log from Sota, approach:', approach)        
    
    elif approach == 'AgentSimulator':
        log_sim = pd.read_csv(f'AgentSimulator/{case_study}/main_results/sim.csv')
        print('imported log from Sota, approach:', approach)    

    elif approach == 'SIMOD':
        log_sim = pd.read_csv(f'SIMOD_TEST/{case_study}/output/best_result/evaluation/simulated_log_0.csv')
        print('imported log from Sota, approach:', approach)

    elif approach == 'DSIM':
        log_sim = pd.read_csv(f'DSIM/results/generated_logs/{case_study}/DSIM/sim.csv')
        print('imported log from Sota, approach:', approach)

    else:
        log_sim = pd.read_csv(f'Sota/{case_study}/{approach}/sim.csv')
    
    #Rename the columns named 'end_timestamp' and 'start_timestamp' with 'time:timestamp' and 'start:timestamp'
    log_sim = log_sim.rename(columns={'end_timestamp': 'time:timestamp', 
                                      'start_timestamp': 'start:timestamp', 
                                      'task':'concept:name',
                                      'caseid': 'case:concept:name',
                                      'resource': 'org:resource'}, errors='ignore')
    
    # Errors='ignore' has been set because the log from francesca meneghello et al has some columns already correctly

    print('imported log from Sota, approach:', approach, ',columns renamed')

parsing log, completed traces ::   0%|          | 0/5771 [00:00<?, ?it/s]

imported log from Sota, approach: SIMOD
imported log from Sota, approach: SIMOD ,columns renamed


In [5]:
# convert event log format lifecycles
if 'lifecycle:transition' in log_real.columns:
    log_real = convert_log(log_real)
    log_real.rename(columns={'START': 'start:timestamp', 'END': 'time:timestamp'}, errors='ignore', inplace=True)
    log_real.reset_index(inplace=True)
if 'lifecycle:transition' in log_sim.columns:
    log_sim = convert_log(log_sim)
    log_sim.reset_index(inplace=True)

100%|██████████| 5771/5771 [01:11<00:00, 80.48it/s]


In [6]:
# Set event log column ID mapping
event_log_ids = EventLogIDs(
    case="case:concept:name",
    activity="concept:name",
    start_time="start:timestamp",
    end_time="time:timestamp",
    resource="org:resource",
)

In [ ]:
log_real[event_log_ids.start_time] = pd.to_datetime(log_real[event_log_ids.start_time], utc=True)
log_real[event_log_ids.end_time] = pd.to_datetime(log_real[event_log_ids.end_time], utc=True)

# log_sim[event_log_ids.start_time] = [i[:19] for i in log_sim[event_log_ids.start_time].values]
# log_sim[event_log_ids.end_time] = [i[:19] for i in log_sim[event_log_ids.end_time].values]
# log_sim[event_log_ids.start_time] = pd.to_datetime(log_sim[event_log_ids.start_time], utc=True)
# log_sim[event_log_ids.end_time] = pd.to_datetime(log_sim[event_log_ids.end_time], utc=True)

In [8]:
#Initialize distances dictionary
distances = {}

In [ ]:
from log_distance_measures.circadian_workforce_distribution import circadian_workforce_distribution_distance
print('case study is: ', case_study, 'CWD is: ', circadian_workforce_distribution_distance(log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
))

### Generateed Attributes Distribution Distance

In [10]:
distance = emd_attributes(log_real, log_sim, attr_names=[])
print('Attr distance: ', distance)
# Fill the distances dictionary
distances['emd_attributes'] = distance

Attr distance:  nan


<b>WARNING: It may take a long time</b>

### Control-flow Log Distance

In [ ]:
# Call passing the event logs, and its column ID mappings
distance = control_flow_log_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
)

In [ ]:
print('CF Log distance: ', distance)

# Fill the distances dictionary
distances['control_flow_log_distance'] = distance


### N-Gram Distribution Distance

In [19]:
n_gram = 3

In [20]:
# Call passing the event logs, and its column ID mappings
distance = n_gram_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    n=n_gram
)

In [ ]:
print('N-Gram distr. distance: ', distance)
# Fill the distances dictionary
distances['n_gram_distribution_distance'] = distance

### Absolute Event Distribution Distance

In [22]:
# EMD of the (END) timestamps distribution where each bin represents a minute
distance = absolute_event_distribution_distance(
    log_real, event_log_ids,
    log_sim, event_log_ids,
    discretize_type=AbsoluteTimestampType.END,
    discretize_event=discretize_to_hour
)

In [ ]:
print('Absolute Event Distribution Distance: ', distance)
# Fill the distances dictionary
distances['absolute_event_distribution_distance'] = distance

### Case Arrival Distribution Distance

In [24]:
distance = case_arrival_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_event=discretize_to_hour  # Function to discretize each timestamp (default by hour)
)

In [ ]:
print('Case Arrival distr distance: ', distance)
# Fill the distances dictionary
distances['case_arrival_distribution_distance'] = distance

### Circadian Event Distribution Distance

In [26]:
distance = circadian_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
)

In [ ]:
print('Circadian Event distr distance: ', distance)
# Fill the distances dictionary 
distances['circadian_event_distribution_distance'] = distance

### Resource-Based Circadian Event Distribution Distance


In [28]:
distance = resource_based_circadian_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
)

In [ ]:
print('Resource-Based Circadian Event distr distance: ', distance)
# Fill the distances dictionary 
distances['resource_based_circadian_event_distribution_distance'] = distance

### Relative Event Distribution Distance

In [30]:
# Call passing the event logs, its column ID mappings, timestamp type, and discretize function
distance = relative_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
    discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
)

In [ ]:
print('Relative Event distr distance: ', distance)
# Fill the distances dictionary
distances['relative_event_distribution_distance'] = distance

### Work in Progress Distance

<b>WARNING: It may take a long time</b>

In [32]:
# # Call passing the event logs, its column ID mappings, timestamp type, and discretize function
# distance = work_in_progress_distance(
#     log_real, event_log_ids,  # First event log and its column id mappings
#     log_sim, event_log_ids,  # Second event log and its column id mappings
#     window_size=pd.Timedelta(hours=1)  # Bins of 1 hour
# )

In [33]:
# print('Work in Progress distance: ', distance)
# Fill the distances dictionary
# distances['work_in_progress_distance'] = distance

### Cycle Time Distribution Distance

In [34]:
distance = cycle_time_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    bin_size=pd.Timedelta(hours=1)  # Bins of 1 minute
)

In [ ]:
print('Cycle Time distr distance: ', distance)
# Fill the distances dictionary
distances['cycle_time_distribution_distance'] = distance

### Save them

In [ ]:
print(f'case study is {case_study} for approach {approach}, with our approach {our_approach} and Sota {Sota}')
# Save the distances dictionary to a file with pkl
pd.to_pickle(distances, f'simulations/{case_study}/metrics.pkl')

# Print the distances vector with each distance measure, in different lines
for key, value in distances.items():
    print(key, value)



### Cast them into .xes for evaluating entropy

In [21]:
import subprocess
import pm4py

merge = False

for case_study in case_studies.values():

    if not merge:
        # Import the .csv
        log = pd.read_csv(f'simulations/{case_study}/sim_0.csv')
        pref_num = len(log)
        log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])
        log['case:concept:name'] = log['case:concept:name'].astype(str)
        log = pm4py.convert_to_event_log(log)
        # log_real = pm4py.read_xes(f'data/{case_study}/logTrain.xes')
    
    else:
        log = pm4py.read_xes(f'data/{case_study}/logTrain.xes')
        #convert the log to a dataframe
        train_original = pm4py.convert_to_dataframe(train_original)
        train_original = train_original[log.columns]  # Ensure the columns match

        # Merge the two logs
        log = pd.concat([train_original, log], ignore_index=True)

    # Save it
    pm4py.write_xes(log, f'simulations/{case_study}/sim_0.xes')

    print(f'Converted {case_study} log to XES format')

    result = subprocess.run(f'java -jar /home/padela/Scrivania/eventropy/eventropy.jar -f /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data/{case_study}/logTrain.xes',
                             shell=True, capture_output=True, text=True)
    print(f"Real - Trace", result.stdout, f'The number of traces is {len(log)}')

    
    
    
    
    result = subprocess.run(f'java -jar /home/padela/Scrivania/eventropy/eventropy.jar -f /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/simulations/{case_study}/sim_0.xes',
                             shell=True, capture_output=True, text=True)
    print(f"Gen - Trace", result.stdout, f'The number of traces is {len(log)}')
    
    
    
    
    
    
    
    result = subprocess.run(f'java -jar /home/padela/Scrivania/eventropy/eventropy.jar -p /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data/{case_study}/logTrain.xes',
                             shell=True, capture_output=True, text=True)
    print(f"Real - Prefix", result.stdout, f'The number of prefixes is {pref_num}')

    
    
    
    
    
    
    result = subprocess.run(f'java -jar /home/padela/Scrivania/eventropy/eventropy.jar -p /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/simulations/{case_study}/sim_0.xes',
                             shell=True, capture_output=True, text=True)
    print(f"Gen - Prefix", result.stdout, f'The number of prefixes is {pref_num}')


exporting log, completed traces ::   0%|          | 0/122 [00:00<?, ?it/s]

Converted Purchasing log to XES format
Real - Trace 
~/logTrain.xes | Trace entropy    :        6.477  
 The number of traces is 122
Gen - Trace 
~/sim_0.xes | Trace entropy    :        5.626  
 The number of traces is 122
Real - Prefix 
~/logTrain.xes | Prefix based entropy :        8.983  
 The number of prefixes is 3760
Gen - Prefix 
~/sim_0.xes | Prefix based entropy :         8.63  
 The number of prefixes is 3760


exporting log, completed traces ::   0%|          | 0/45 [00:00<?, ?it/s]

Converted Production log to XES format
Real - Trace 
~/logTrain.xes | Trace entropy    :        7.481  
 The number of traces is 45
Gen - Trace 
~/sim_0.xes | Trace entropy    :        5.447  
 The number of traces is 45
Real - Prefix 
~/logTrain.xes | Prefix based entropy :       12.608  
 The number of prefixes is 2484
Gen - Prefix 
~/sim_0.xes | Prefix based entropy :       11.112  
 The number of prefixes is 2484


exporting log, completed traces ::   0%|          | 0/191 [00:00<?, ?it/s]

...

Converted sepsis log to XES format
Real - Trace  The number of traces is 210
Gen - Trace 
~/sim_0.xes | Trace entropy    :        7.535  
 The number of traces is 210
Real - Prefix  The number of prefixes is 2983
Gen - Prefix 
~/sim_0.xes | Prefix based entropy :        9.378  
 The number of prefixes is 2983


exporting log, completed traces ::   0%|          | 0/30074 [00:00<?, ?it/s]

Converted rtf log to XES format
Real - Trace  The number of traces is 30074
Gen - Trace  The number of traces is 30074
Real - Prefix  The number of prefixes is 113831
Gen - Prefix  The number of prefixes is 113831


exporting log, completed traces ::   0%|          | 0/13437 [00:00<?, ?it/s]

Converted bpi19 log to XES format
Real - Trace 
~/logTrain.xes | Trace entropy    :        5.993  
 The number of traces is 13437
Gen - Trace 
~/sim_0.xes | Trace entropy    :        6.072  
 The number of traces is 13437
Real - Prefix 
~/logTrain.xes | Prefix based entropy :        7.837  
 The number of prefixes is 69317
Gen - Prefix 
~/sim_0.xes | Prefix based entropy :        7.606  
 The number of prefixes is 69317


In [10]:
case_studies

{1: 'Purchasing',
 2: 'Production',
 3: 'Consulta',
 4: 'bpi12',
 5: 'bpi17',
 6: 'sepsis',
 7: 'rtf',
 8: 'bpi19'}